<a href="https://colab.research.google.com/github/mayankshouche/DSLabFinalProject/blob/main/10000_BERT_Frozen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls 

drive  sample_data


In [4]:
import pandas as pd
X_train = pd.read_csv("/content/drive/MyDrive/X_train.csv.zip")
X_val = pd.read_csv("/content/drive/MyDrive/X_val.csv.zip")
y_train = pd.read_csv("/content/drive/MyDrive/y_train.csv.zip")
y_val = pd.read_csv("/content/drive/MyDrive/y_val.csv.zip")

In [5]:
train = [X_train, y_train['party']]
dataset = pd.concat(train, axis=1)
dataset = dataset[dataset["party"].isin(["R","D"])]

dataset_small = dataset.sample(frac=0.33)

dataset_small.head()
dataset = dataset_small
len(dataset)

156197

In [6]:
test = [X_val, y_val['party']]
test_dataset = pd.concat(test, axis=1)
test_dataset = test_dataset[test_dataset["party"].isin(["R","D"])]
test_dataset.head()
test_small = dataset.sample(frac=0.4)
test_dataset = test_small
len(test_dataset)

62479

In [7]:
# !pip install tweet-preprocessor
# import preprocessor as p
import re 

def preprocess_tweet(tweet):
  # tweet = tweet['text']
  clean_tweet = tweet.replace('@', '')
  clean_tweet = clean_tweet.replace('#', '')
  clean_tweet = clean_tweet.replace('&', '')
  clean_tweet = re.sub(r'[^A-Za-z0-9.!, ]+', '', clean_tweet)
  split = [word.lower() for word in clean_tweet.split() if 'http' not in word.lower() and 'jpg' not in word.lower() and 'www' not in word.lower() and word.lower() not in['amp', 'qt']]
  return ' '.join(split)
  # print(res)
  # return res


dataset["clean_text"] = dataset['text'].apply(lambda x: preprocess_tweet(x))
test_dataset["clean_text"] = test_dataset['text'].apply(lambda x: preprocess_tweet(x))


In [8]:
dataset["label"] = dataset['party'].apply(lambda party: 1 if party=='R' else 0)
test_dataset["label"] = test_dataset['party'].apply(lambda party: 1 if party=='R' else 0)

In [9]:
!pip install transformers datasets 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wordcloud
# import preprocessor as p # tweet-preprocessor
import nltk
import re
import seaborn as sns
import torch

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from datasets import Dataset
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

     |████████████████████████████████| 1.4MB 12.9MB/s 
     |████████████████████████████████| 163kB 53.5MB/s 
     |████████████████████████████████| 890kB 54.4MB/s 
     |████████████████████████████████| 2.9MB 54.4MB/s 
     |████████████████████████████████| 245kB 60.1MB/s 
     |████████████████████████████████| 17.7MB 177kB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=debccd9e5e228e40a9af7463b4e26d6f859a73db75600fddf452c439220e1774
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [12]:
device = torch.device("cuda")

In [13]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments


# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
#                                           do_lower_case=True)

In [14]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    dataset.clean_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True,
    truncation=True,
    return_tensors='pt',
)

encoded_data_test = tokenizer.batch_encode_plus(
    test_dataset.clean_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    truncation=True,
    return_tensors='pt',
)
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(dataset.label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test_dataset.label.values)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
test_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)

BATCH_SIZE = 8

train_dataloader = DataLoader(train_data, 
                              sampler=RandomSampler(train_data), 
                              batch_size=BATCH_SIZE)

test_dataloader = DataLoader(test_data,
                             sampler=SequentialSampler(test_data),
                             batch_size=BATCH_SIZE)


In [16]:
EPOCHS = 5

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, 
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [17]:
dataset

,Unnamed: 0,Unnamed: 0.1,id,screen_name,user_id,time,link,text,source,party,clean_text,label
166548,246884,1425,1258816596707082240,SenJeffMerkley,29201047,2020-05-08T13:50:38-04:00,https://www.twitter.com/votesolar/statuses/125...,RT @votesolar Our new analysis found that as a...,Twitter Web App,D,rt votesolar our new analysis found that as a ...,0
138541,67909,623,1227211860941910016,DemCloakroom,2557507771,2020-02-11T07:44:42-05:00,https://www.twitter.com/DemCloakroom/statuses/...,"The House will meet today - Tuesday, February ...",Twitter Web App,D,"the house will meet today tuesday, february 11...",0
356061,56201,6553,1224886034913288192,RepJimmyGomez,2371339658,2020-02-04T21:42:42-05:00,https://www.twitter.com/RepJimmyGomez/statuses...,Don't be fooled: @realDonaldTrump is NOT looki...,Twitter Web App,D,dont be fooled realdonaldtrump is not looking ...,0
349929,209941,1167,1252739839432298496,RepDwightEvans,90639372,2020-04-21T19:23:47-04:00,https://www.twitter.com/RepDwightEvans/statuse...,"As a strong advocate of funding our hospitals,...",Twitter for iPhone,D,"as a strong advocate of funding our hospitals,...",0
220692,356593,1695,1280954569078575104,SenatorLoeffler,1200451909406121984,2020-07-08T15:59:02-04:00,https://www.twitter.com/SenatorLoeffler/status...,Welcome to @lopezobrador_. He's been a great p...,Twitter for iPhone,R,welcome to lopezobrador. hes been a great part...,1
...,...,...,...,...,...,...,...,...,...,...,...,...
284550,497546,1132,1310344872851910656,RepEliotEngel,164007407,2020-09-27T18:25:37-04:00,https://www.twitter.com/RepEliotEngel/statuses...,"To all my friends celebrating in #NY16, have a...",Twitter for iPhone,D,"to all my friends celebrating in ny16, have a ...",0
247142,510304,1221,1312857558646956032,ChrisVanHollen,18137749,2020-10-04T16:50:08-04:00,https://www.twitter.com/ChrisVanHollen/statuse...,Let’s get this straight: It’s too reckless for...,Twitter for iPhone,D,lets get this straight its too reckless for th...,0
471908,238245,591,1257868410450587648,RepDavidRouzer,834069080,2020-05-05T23:02:53-04:00,https://www.twitter.com/WECTWeather/statuses/1...,RT @WECTWeather 10:46 p.m. 5/5/20-Intense line...,TweetDeck,R,rt wectweather 1046 p.m. 5520intense line of t...,1
126805,477424,1624,1306730033717878784,RepDelBene,995193054,2020-09-17T19:01:32-04:00,https://www.twitter.com/RepDelBene/statuses/13...,"250,000 pregnant workers are denied reasonable...",Twitter Web App,D,"250,000 pregnant workers are denied reasonable...",0


In [18]:
def auc_score(preds, labels):
  soft_preds = softmax(preds, axis=1) # logit -> probability
  print(preds.shape, labels.shape)
  if np.shape(preds)[1] > 2: # check for multi-class
    return roc_auc_score(labels, soft_preds, multi_class='ovr')
  else:
    soft_preds = soft_preds[:,1]
    return roc_auc_score(labels, soft_preds)

def acc_score_by_class(preds, labels):
  LABEL_MAP = {
      "D": 0,
      "R": 1
  }
  label_dict_inverse = {v: k for k, v in LABEL_MAP.items()} 

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [19]:
def evaluate(model, dataloader):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  
  for batch in dataloader:
      
      # convert data to CUDA
      batch = tuple(b.to(device) for b in batch)
      
      inputs = {
          'input_ids':      batch[0],
          'attention_mask': batch[1],
          'labels':         batch[2],
      }

      with torch.no_grad():        
          outputs = model(**inputs) # get predictions
          
      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)
  
  loss_val_avg = loss_val_total/len(dataloader) 
  
  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
          
  return loss_val_avg, predictions, true_vals

In [ ]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [22]:
print(device)
torch.cuda.get_device_name(0)

cuda


'Tesla T4'

In [23]:
def train(model, epochs, train_dataloader, test_dataloader, optimizer, scheduler):
  losses_train = []
  losses_test = []
  aucs_test = []

  for epoch in tqdm(range(1, epochs+1)): # use tqdm for a progress bar
    model.train() # enter training mode
    loss_train_total = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch}', leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
        
        # get CUDA data
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'labels':         batch[2],
        }

        outputs = model(**inputs) # evaluate
        
        # for reference, we are using cross-entropy loss here,
        # as implemented in https://huggingface.co/transformers/_modules/transformers/modeling_bert.html
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward() # do backprop

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
          
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dataloader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    losses_train.append(loss_train_avg)
    
    val_loss, predictions, true_vals = evaluate(model, test_dataloader)
    auc = auc_score(predictions, true_vals)
    losses_test.append(val_loss)
    aucs_test.append(auc)
    tqdm.write(f'Testing loss: {val_loss}')
    tqdm.write(f'AUC: {auc}')
    
  return losses_train, losses_test, aucs_test

In [24]:
EPOCHS = 2

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, 
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [26]:
for name, param in model.named_parameters():
	if 'classifier' not in name: # classifier layer
		param.requires_grad = False

losses_train, losses_test, aucs_test = train(model, 
                                             EPOCHS, 
                                             train_dataloader, 
                                             test_dataloader, 
                                             optimizer, 
                                             scheduler)



Epoch 1
Training loss: 0.6664756098462128
(62479, 2) (62479,)
Testing loss: 0.6591791033439539
AUC: 0.5936736950720791


KeyboardInterrupt: ignored